<a href="https://colab.research.google.com/github/thanga-v2/Transformers/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Two types of Summarization

- Extractive
- Abstractive

Extractive summaries are build out of the documents itself. This summary is a subset of the document.

Abstractive summaries, can contain sequences which are not present in the document.



In [1]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2024-08-21 09:47:40--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.08s   

2024-08-21 09:47:41 (62.0 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [3]:
import pandas as pd
import numpy as np
import textwrap
from transformers import pipeline

In [4]:
data = pd.read_csv('bbc_text_cls.csv')

In [32]:
data.head(10)

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business
5,Japan narrowly escapes recession\n\nJapan's ec...,business
6,Jobs growth still slow in the US\n\nThe US cre...,business
7,"India calls for fair trade rules\n\nIndia, whi...",business
8,Ethiopia's crop production up 24%\n\nEthiopia ...,business
9,Court rejects $280bn tobacco case\n\nA US gove...,business


In [6]:
data.columns

Index(['text', 'labels'], dtype='object')

In [9]:
data['labels'].unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [15]:
x=data['labels'].groupby(data['labels'])

In [20]:
x.count()

,labels
labels,
business,510
entertainment,386
politics,417
sport,511
tech,401


In [25]:
type(x)

pandas.core.groupby.generic.SeriesGroupBy

In [28]:
v=data.groupby(data['labels'])

In [29]:
type(v)

pandas.core.groupby.generic.DataFrameGroupBy

In [30]:
v.count()

,text
labels,
business,510
entertainment,386
politics,417
sport,511
tech,401


from matplotlib import pyplot as plt
_df_0['text'].plot(kind='hist', bins=20, title='text')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['text'].plot(kind='line', figsize=(8, 4), title='text')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_2['text'].plot(kind='hist', bins=20, title='text')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('labels').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4['text'].plot(kind='line', figsize=(8, 4), title='text')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['labels'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='text', y='labels', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [44]:
data[data.labels == 'business']['text']

,text
0,Ad sales boost Time Warner profit\n\nQuarterly...
1,Dollar gains on Greenspan speech\n\nThe dollar...
2,Yukos unit buyer faces loan claim\n\nThe owner...
3,High fuel prices hit BA's profits\n\nBritish A...
4,Pernod takeover talk lifts Domecq\n\nShares in...
...,...
505,Trial begins of Spain's top banker\n\nThe tria...
506,UK economy ends year with spurt\n\nThe UK econ...
507,HealthSouth ex-boss goes on trial\n\nThe forme...
508,Euro firms miss out on optimism\n\nMore than 9...


In [34]:
data[data['labels'] == 'business']

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business
...,...,...
505,Trial begins of Spain's top banker\n\nThe tria...,business
506,UK economy ends year with spurt\n\nThe UK econ...,business
507,HealthSouth ex-boss goes on trial\n\nThe forme...,business
508,Euro firms miss out on optimism\n\nMore than 9...,business


In [52]:
data[data.labels == 'business']['text'].sample()

,text
351,Share boost for feud-hit Reliance\n\nThe board...


In [59]:
data[data.labels == 'business']['text'].sample(random_state=42)

,text
480,Christmas sales worst since 1981\n\nUK retail ...


In [60]:
doc = data[data.labels == 'business']['text'].sample(random_state=42)

In [61]:
doc

,text
480,Christmas sales worst since 1981\n\nUK retail ...


In [62]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [68]:
doc.iloc[0]

'Christmas sales worst since 1981\n\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.\n\nRetail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.\n\nThe last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.\n\nThe ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The N

In [72]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [71]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

**Distil Bart** - Example for seq to seq models.

such is good for **language translations**.

means, outut is not necessarily equal to the length of the input.

In [82]:
# remove the first line as it might also be a title.


doc.iloc[0].split('\n',1)

['Christmas sales worst since 1981',
 '\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.\n\nRetail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.\n\nThe last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.\n\nThe ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. T

In [83]:
print(doc.iloc[0].split('\n',1)[1])


UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.

Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.

The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.

The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The November-December jump last year was roughl

In [87]:
summarizer(doc.iloc[0].split('\n',1)[1])

[{'summary_text': ' Retail sales dropped by 1% on the month in December, after a 0.6% rise in November . Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth . The last time retailers endured a tougher Christmas was 23 years ago, when sales plunged 1.7% .'}]

In [94]:
def print_summary(doc):
  result = summarizer(doc.iloc[0].split('\n',1)[1])
  print(wrap(result[0]['summary_text']))

In [89]:
result = summarizer(doc.iloc[0].split('\n',1)[1])

In [92]:
result[0]['summary_text']

' Retail sales dropped by 1% on the month in December, after a 0.6% rise in November . Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth . The last time retailers endured a tougher Christmas was 23 years ago, when sales plunged 1.7% .'

In [93]:
print(wrap(result[0]['summary_text']))

 Retail sales dropped by 1% on the month in December, after a 0.6%
rise in November . Clothing retailers and non-specialist stores were
the worst hit with only internet retailers showing any significant
growth . The last time retailers endured a tougher Christmas was 23
years ago, when sales plunged 1.7% .


In [102]:
doc.iloc[0]

'Christmas sales worst since 1981\n\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.\n\nRetail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.\n\nThe last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.\n\nThe ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The N

In [103]:
print_summary(doc)

 Retail sales dropped by 1% on the month in December, after a 0.6%
rise in November . Clothing retailers and non-specialist stores were
the worst hit with only internet retailers showing any significant
growth . The last time retailers endured a tougher Christmas was 23
years ago, when sales plunged 1.7% .


Lets select an other random article from the entrtainment label

In [117]:
ent_data = data[data['labels'] == 'entertainment']['text'].sample(random_state=123)

In [122]:
ent_data

,text
795,Goodrem wins top female MTV prize\n\nPop singe...


In [123]:
type(ent_data)

pandas.core.series.Series

In [124]:
ent_data.iloc[0]

"Goodrem wins top female MTV prize\n\nPop singer Delta Goodrem has scooped one of the top individual prizes at the first Australian MTV Music Awards.\n\nThe 21-year-old singer won the award for best female artist, with Australian Idol runner-up Shannon Noll taking the title of best male at the ceremony. Goodrem, known in both Britain and Australia for her role as Nina Tucker in TV soap Neighbours, also performed a duet with boyfriend Brian McFadden. Other winners included Green Day, voted best group, and the Black Eyed Peas. Goodrem, Green Day and the Black Eyed Peas took home two awards each. As well as best female, Goodrem also took home the Pepsi Viewers Choice Award, whilst Green Day bagged the prize for best rock video for American Idiot. The Black Eyed Peas won awards for best R 'n' B video and sexiest video, both for Hey Mama. Local singer and songwriter Missy Higgins took the title of breakthrough artist of the year, with Australian Idol winner Guy Sebastian taking the honours 

In [125]:
wrap(ent_data.iloc[0])

"Goodrem wins top female MTV prize\n\nPop singer Delta Goodrem has\nscooped one of the top individual prizes at the first Australian MTV\nMusic Awards.\n\nThe 21-year-old singer won the award for best female\nartist, with Australian Idol runner-up Shannon Noll taking the title\nof best male at the ceremony.  Goodrem, known in both Britain and\nAustralia for her role as Nina Tucker in TV soap Neighbours, also\nperformed a duet with boyfriend Brian McFadden.  Other winners\nincluded Green Day, voted best group, and the Black Eyed Peas.\nGoodrem, Green Day and the Black Eyed Peas took home two awards each.\nAs well as best female, Goodrem also took home the Pepsi Viewers\nChoice Award, whilst Green Day bagged the prize for best rock video\nfor American Idiot.  The Black Eyed Peas won awards for best R 'n' B\nvideo and sexiest video, both for Hey Mama.  Local singer and\nsongwriter Missy Higgins took the title of breakthrough artist of the\nyear, with Australian Idol winner Guy Sebastian t

In [126]:
print(wrap(ent_data.iloc[0]))

Goodrem wins top female MTV prize

Pop singer Delta Goodrem has
scooped one of the top individual prizes at the first Australian MTV
Music Awards.

The 21-year-old singer won the award for best female
artist, with Australian Idol runner-up Shannon Noll taking the title
of best male at the ceremony.  Goodrem, known in both Britain and
Australia for her role as Nina Tucker in TV soap Neighbours, also
performed a duet with boyfriend Brian McFadden.  Other winners
included Green Day, voted best group, and the Black Eyed Peas.
Goodrem, Green Day and the Black Eyed Peas took home two awards each.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.  The Black Eyed Peas won awards for best R 'n' B
video and sexiest video, both for Hey Mama.  Local singer and
songwriter Missy Higgins took the title of breakthrough artist of the
year, with Australian Idol winner Guy Sebastian taking the honours f

In [127]:
print_summary(ent_data)

 The 21-year-old singer won the award for best female artist .
Australian Idol runner-up Shannon Noll took the title of best male at
the ceremony . Other winners included Green Day, the Black Eyed Peas,
Missy Higgins and Green Day . The VH1 First Music Award went to Cher
honouring her achievements within the music industry .
